pip install -U sentence-transformers

In [1]:
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import os

In [2]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# Define Input

In [3]:
background = "I'm interested in the AI applied in robotics."
folder_path = "synthetic-documents"

# Get Matrix

We don't incorporate get_matrix into a tool because the matrix is fixed once the documents are finished. So that the matrix will not be initialized every time we kick off the crew (or when new query comes in).

If you want to train with SBERT model, run the following cell instead.

# Creating Tools

In [4]:
# Use tf-idf version
from TFIDFActivityRecommendationTool import TFIDFActivityRecommendationTool
activity_recommendation_tool = TFIDFActivityRecommendationTool()

# OR
# Use SBERT version
#from SBERTActivityRecommendationTool import SBERTActivityRecommendationTool
#activity_recommendation_tool = SBERTActivityRecommendationTool()

from FileReadTool import FileReadTool
file_read_tool = FileReadTool()

# Creating Agents

In [5]:
recommendation_agent = Agent(
    role="Activity Recommendation Researcher",
    goal="Help do the information analysis of all activities and find the most"
        "useful ones for guests at European Robitics Forum",
    backstory=(f"""
        With extensive knowledge of the events, you are equipped to provide accurate information 
        and valuable insights.
        Using a specialized tool, you can tailor recommendations for activities that best fit 
        each guest's background and interests.
        
        The guest's background is:
        {background}
        
        Please don't recommend anything else except the ones you obtain with the tool.
        You can modify the description of these activities, making it more straightforward
        and concise.
        """
    ),
    tools = [activity_recommendation_tool, file_read_tool],
    llm=ChatOpenAI(model='gpt-3.5-turbo'),
    allow_delegation=False,
    verbose=False
)

In [6]:
guest_guide_bot_agent = Agent(
    role="Guest Guide Bot",
    goal="Be a professional "
        "guest guide volunteer to assist guests at European Robitics Forum",
    backstory=(f"""
        As a dedicated bot for the European Robotics Forum, 
        you are passionate about robotics and committed to enhancing the experience of every guest.
        You should convert the structured information from the last task to a chatbot style output in
        order to interact with guests friendly.
        
        The guest's background is:
        {background}
        
        Please don't make up any informtion and only use the information from the previous task.
        Your friendly demeanor and willingness to assist ensure that each guest feels welcome 
        and supported, making their visit both enjoyable and informative.
        Your role is vital in fostering a positive atmosphere and ensuring the smooth operation 
        of the forum.
        """
    ),
    tools = [activity_recommendation_tool],
    llm=ChatOpenAI(model='gpt-3.5-turbo'),
    allow_delegation=False,
    verbose=False
)

# Creating Tasks

In [7]:
provide_recommendations_task = Task(
    description=(
        f"guest just reached out with a request for activity recommendations."
        f"guest has a background in {background}. "
        f"Make sure to use the specialized tool to find activities that best fit their interests and background. "
        f"Strive to provide a complete and accurate recommended activities."
        f"The description should be filled with the summarization of the contents."
    ),
    expected_output=("""
        output = [
                        {
                            "file_name": "synthetic-documents/abc.docx",
                            "topic": "abc",
                            "contents": "hhh"
                            "description": "abh"
                        },
                        {
                            "file_name": "synthetic-documents/xxx.docx",
                            "topic": "dec",
                            "contents": "hhh"
                            "description": "dse"
                        },
                        # Add more dictionaries as needed
                ]"""      
    ),
    agent=recommendation_agent,
    tool=[activity_recommendation_tool, file_read_tool]
)

In [8]:
chatbot_output_task  = Task(
    description=(
        """Convert the output from the previous task into a chatbot style output.
        Ignore the fields "file_name" and "contents",  the output must contain "topic" and information from "description". 
        It's super important that you don't make up things and you are faithful to the information from the last task.
        """
    ),
    expected_output=("""A chatbot style generation.
    For example:
    Welcome to European Robotics Forum! Based on your profile as a professor interested in ...I recommend the following activities for you
    ...
    Hope you enjoy the time here!
    """
     ),
    agent=guest_guide_bot_agent,
    tool=[]
)

# Create the Crew

In [9]:
crew = Crew(
  agents=[recommendation_agent,guest_guide_bot_agent],
  tasks=[provide_recommendations_task, chatbot_output_task],
  verbose=True,
  memory=True
)

# Run the Crew

In [10]:
result = crew.kickoff()

 [2024-10-23 12:23:00][DEBUG]: == Working Agent: Activity Recommendation Researcher
 [2024-10-23 12:23:00][INFO]: == Starting Task: guest just reached out with a request for activity recommendations.guest has a background in I'm interested in the AI applied in robotics.. Make sure to use the specialized tool to find activities that best fit their interests and background. Strive to provide a complete and accurate recommended activities.The description should be filled with the summarization of the contents.


Inserting batches in chromadb: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.16it/s]


 [2024-10-23 12:23:09][DEBUG]: == [Activity Recommendation Researcher] Task output: [
    {
        "file_name": "synthetic-documents/workshop4.docx",
        "topic": "Next-Gen Robotics Technologies: Robotics and AI",
        "contents": "Welcome to 'Next-Gen Robotics Technologies: Robotics and AI,' a dynamic workshop designed for technologists, engineers, researchers, and innovators who are eager to delve into the convergence of robotics and artificial intelligence. This session will explore cutting-edge advancements and future directions in robotic technology, emphasizing the integration of AI to create more sophisticated, autonomous systems.",
        "description": "A workshop focusing on the convergence of robotics and artificial intelligence, covering topics such as foundations of AI in robotics, case studies of AI-driven robotics, hands-on AI robotics labs, ethical and societal implications, innovation and future trends, and a panel discussion with industry leaders."
    },
   

Inserting batches in chromadb: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.08it/s]


 [2024-10-23 12:23:23][DEBUG]: == [Guest Guide Bot] Task output: Welcome to the European Robotics Forum! Based on your interest in AI applied in robotics, I recommend the following activities for you:

1. Workshop: Next-Gen Robotics Technologies: Robotics and AI
   - Description: This workshop explores the convergence of robotics and artificial intelligence, covering topics such as foundations of AI in robotics, case studies of AI-driven robotics, hands-on AI robotics labs, ethical and societal implications, innovation, future trends, and a panel discussion with industry leaders.
   
2. Exhibition: Regulatory and Legal Landscape & Standardization: AI Act and Robotics
   - Description: This exhibition focuses on the regulatory and legal frameworks related to robotics and artificial intelligence, including AI legislation, global standards in robotics, interactive seminars, panel discussions, compliance tools, and networking opportunities.
   
3. Workshop: Robotics in Key Application Area

In [11]:
from IPython.display import Markdown, display
display(Markdown(result.raw))

Welcome to the European Robotics Forum! Based on your interest in AI applied in robotics, I recommend the following activities for you:

1. Workshop: Next-Gen Robotics Technologies: Robotics and AI
   - Description: This workshop explores the convergence of robotics and artificial intelligence, covering topics such as foundations of AI in robotics, case studies of AI-driven robotics, hands-on AI robotics labs, ethical and societal implications, innovation, future trends, and a panel discussion with industry leaders.
   
2. Exhibition: Regulatory and Legal Landscape & Standardization: AI Act and Robotics
   - Description: This exhibition focuses on the regulatory and legal frameworks related to robotics and artificial intelligence, including AI legislation, global standards in robotics, interactive seminars, panel discussions, compliance tools, and networking opportunities.
   
3. Workshop: Robotics in Key Application Areas: Inspection and Maintenance
   - Description: This workshop delves into the application of robotics in inspection and maintenance processes, covering technological innovations, AI and predictive maintenance, case studies, hands-on demonstrations, safety, compliance, and roundtable discussions.

Hope you enjoy the time here at the European Robotics Forum!

Inserting batches in chromadb: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.29it/s]
